In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, './..')
sys.path.insert(0, '../data')

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import proplot as pplt
import pandas as pd

from tqdm import tqdm
import numpy as np
import torch
from torchvision import datasets, transforms

from models import eval
from models import model as model_loader
import plots as pl
from utils import dev, load_data, classification, make_orth_basis
import experiment_parser as exp
from curve_utils import tab_name_to_hex, load_mnist, load_cifar

sys.path.insert(0, './../../')

import response_contour_analysis.utils.model_handling as model_utils
import response_contour_analysis.utils.dataset_generation as data_utils
import response_contour_analysis.utils.histogram_analysis as hist_utils
import response_contour_analysis.utils.principal_curvature as curve_utils
import response_contour_analysis.utils.plotting as plot_utils

# check device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

# Experiment parameters

In [ ]:
dataset_type = 0
num_images = 20

In [ ]:
plot_settings = {
        "text.usetex": True,
        "font.family": "serif",
        "font.size": 8,
        "axes.formatter.use_mathtext":True,
}
pplt.rc.update(plot_settings)
mpl.rcParams.update(plot_settings)

figwidth = '13.968cm'
figwidth_inch = 5.50107 
dpi = 600
model_types = ['Naturally trained', 'Adversarially trained']
plot_colors = [tab_name_to_hex('tab:blue'), tab_name_to_hex('tab:red')]

# Load data

In [ ]:
if dataset_type == 0: # MNIST
    model_natural, data_natural, model_robust, data_robust = load_mnist(code_directory='../../', seed=0)
else: # CIFAR
    model_natural, data_natural, model_robust, data_robust = load_cifar(code_directory='../../')

run_outputs = []
all_origin_indices = []
adv_origin_indices = []
for run_type in range(7):
    run_outputs.append(exp.get_combined_experiment_outputs(dataset_type, run_type, num_images))

num_advs = run_outputs[0][0].shape[1]
adv_origin_indices = exp.get_index_data(dataset_type, 0, num_images, num_advs)[1]

data_paired_natural = adv_origin_indices['data_natural_paired']
data_paired_madry = adv_origin_indices['data_robust_paired']

paired_principal_curvatures = np.stack([
    run_outputs[0]['principal_curvatures'], # natural, dataset pair
    run_outputs[1]['principal_curvatures']], axis=0) # robust, dataset pair
paired_principal_directions = np.stack([
    run_outputs[0]['principal_directions'], # natural, dataset pair
    run_outputs[1]['principal_directions']], axis=0) # robust, dataset pair
paired_mean_curvatures = np.mean(paired_principal_curvatures, axis=-1)
paired_origin_indices = np.stack([
    run_outputs[0]['origin_indices'], # natural, dataset pair
    run_outputs[1]['origin_indices']], axis=0) #robust, dataset pair

adv_principal_curvatures = np.stack([
    run_outputs[2]['principal_curvatures'], # natural, adversarial pair
    run_outputs[3]['principal_curvatures']], axis=0) # robust, adversarial pair
adv_principal_directions = np.stack([
    run_outputs[2]['principal_directions'], # natural, adversarial pair
    run_outputs[3]['principal_directions']], axis=0) # robust, adversarial pair
adv_mean_curvatures = np.mean(adv_principal_curvatures, axis=-1)
adv_origin_indices = np.stack([
    run_outputs[2]['origin_indices'], # natural, adversarial pair
    run_outputs[3]['origin_indices']], axis=0) # robust, adversarial pair

rand_subspace_pcs = np.stack([
    run_outputs[4]['principal_curvatures'], # natural, random subspace
    run_outputs[5]['principal_curvatures']], axis=0) # robust, random subspace
rand_subspace_pds = np.stack([
    run_outputs[4]['principal_directions'], # natural, random subspace
    run_outputs[5]['principal_directions']], axis=0) # robust, random subspace
    
adv_subspace_pcs = np.stack([
    run_outputs[6]['principal_curvatures'], # natural, adversarial subspace
    run_outputs[7]['principal_curvatures']], axis=0) # robust, adversarial subspace
adv_subspace_pds = np.stack([
    run_outputs[6]['principal_directions'], # natural, adversarial subspace
    run_outputs[7]['principal_directions']], axis=0) # robust, adversarial subspace

In [ ]:
bar_width = 0.5
fig, axs = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12,7))
fig.subplots_adjust(top=0.8)

for data_idx, mean_curvatures in enumerate([paired_mean_curvatures, adv_mean_curvatures]):
    for model_idx in range(2):
        boxprops = dict(color=plot_colors[model_idx], linewidth=1.5, alpha=0.7)
        whiskerprops = dict(color=plot_colors[model_idx], alpha=0.7)
        capprops = dict(color=plot_colors[model_idx], alpha=0.7)
        medianprops = dict(linestyle='--', linewidth=0.5, color=plot_colors[model_idx])
        meanpointprops = dict(marker='o', markeredgecolor='black',
                              markerfacecolor=plot_colors[model_idx])
        meanprops = dict(linestyle='-', linewidth=0.5, color=plot_colors[model_idx])
        data = mean_curvatures[model_idx, :, :].reshape(-1)
        axs[data_idx].boxplot(data, sym='', positions=[model_idx], whis=(10, 90), widths=bar_width, meanline=True, showmeans=True, boxprops=boxprops,
            whiskerprops=whiskerprops, capprops=capprops, medianprops=medianprops, meanprops=meanprops)
    axs[data_idx].set_xticks([0, 1], minor=False)
    axs[data_idx].set_xticks([], minor=True)
    axs[data_idx].set_xticklabels(model_types)
    if data_idx == 0:
        axs[data_idx].set_ylabel('Mean curvature')
        axs[data_idx].set_title('Paired image boundary')
    else:
        axs[data_idx].set_title('Adversarial image boundary')

for model_idx in range(adv_mean_curvatures.shape[0]):
    boxprops = dict(color=plot_colors[model_idx], linewidth=1.5, alpha=0.7)
    whiskerprops = dict(color=plot_colors[model_idx], alpha=0.7)
    capprops = dict(color=plot_colors[model_idx], alpha=0.7)
    medianprops = dict(linestyle='--', linewidth=0.5, color=plot_colors[model_idx])
    meanpointprops = dict(marker='o', markeredgecolor='black',
                          markerfacecolor=plot_colors[model_idx])
    meanprops = dict(linestyle='-', linewidth=0.5, color=plot_colors[model_idx])
    for adv_idx in range(adv_mean_curvatures.shape[-1]):
        data = adv_mean_curvatures[model_idx, :, adv_idx].reshape(-1)
        axs[2].boxplot(data, sym='', positions=[adv_idx], whis=(10, 90), widths=bar_width, meanline=True, showmeans=True, boxprops=boxprops,
            whiskerprops=whiskerprops, capprops=capprops, medianprops=medianprops, meanprops=meanprops)
axs[2].set_title('Adversarial image boundary')
axs[2].set_xlabel('Dimension number')
axs[2].set_xticks([i for i in range(adv_mean_curvatures.shape[-1])], minor=False)
axs[2].set_xticks([], minor=True)
axs[2].set_xticklabels([str(i+1) for i in range(adv_mean_curvatures.shape[-1])])

def make_space_above(axes, topmargin=1):
    """ increase figure size to make topmargin (in inches) space for 
        titles, without changing the axes sizes
        obtained from: https://stackoverflow.com/a/55768955/
    """
    fig = axes.flatten()[0].figure
    s = fig.subplotpars
    w, h = fig.get_size_inches()

    figh = h - (1 - s.top) * h + topmargin
    fig.subplots_adjust(bottom=s.bottom*h/figh, top=1-topmargin/figh)
    fig.set_figheight(figh)

make_space_above(axs, topmargin=0.5)  

fig.suptitle(f'Curvature at the decision boundary\nfor {num_images} images and the first {num_advs} adversarial directions', y=1.0)
plt.show()
#fig.savefig('../data/mean_curvature_boxplots.png', transparent=True, bbox_inches='tight', pad_inches=0.01)

In [ ]:
bar_width = 0.5
fig, axs = pplt.subplots(nrows=1, ncols=2, sharey=True, figwidth=figwidth)
titles = ['Test image boundary', 'Adversarial image boundary']
for data_idx, mean_curvatures in enumerate([paired_mean_curvatures, adv_mean_curvatures]):
    data = pd.DataFrame(mean_curvatures.reshape(-1, np.prod(mean_curvatures.shape[1:])).transpose(1, 0),
                        columns=pd.Index(model_types, name=''))
    axs[data_idx].boxplot(data, fill=True, mean=True,
                          cycle=pplt.Cycle(plot_colors),
                          linewidth=0.5,
                          meanlinestyle='-', medianlinestyle='--',
                          marker='o', markersize=1.0
                         )
    axs[data_idx].format(
        xticklabels=model_types,
        ylabel='Mean curvature',
        title=titles[data_idx],
        xgrid=False
    )
    axs[data_idx].axhline(0.0, color='black', linestyle='dashed', linewidth=0.5)


axs.format(
    suptitle=f'Curvature at the decision boundary',#\naveraged across {num_images} images and the first {num_advs} adversarial directions'
)

pplt.show()
#fig.savefig('../data/mean_curvature_boxplots.png', transparent=True, bbox_inches='tight', pad_inches=0.01, dpi=dpi)

In [ ]:
num_models, num_images, num_advs, num_dims = adv_principal_curvatures.shape

fig, ax = pplt.subplots(nrows=1, ncols=1, figwidth=figwidth_inch/2, dpi=dpi, sharey=False, sharex=False)
for image_idx in range(num_images):
    for adv_idx in range(num_advs):
        ax.scatter(adv_principal_curvatures[0, image_idx, adv_idx, :],
                   s=0.01, c=plot_colors[0])
        ax.scatter(adv_principal_curvatures[1, image_idx, adv_idx, :],
                   s=0.01, c=plot_colors[1])
        
ix = ax.inset(
    bounds=[200, 0.50, 400, 1.5],
    transform='data', zoom=True,
    zoom_kw={'edgecolor': 'k', 'lw': 1, 'ls': '--'}
)
ix.format(
    xlim=(0, num_dims), ylim=(-0.02, 0.02), metacolor='red7',
    grid=False,
    linewidth=1.5, ticklabelweight='bold'
)
ix.plot([0, num_dims], [0, 0], lw=0.1, c='k')
ix.scatter(adv_principal_curvatures[0, ...].mean(axis=(0, 1)),
           s=0.005, alpha=1.0, c=plot_colors[0])
ix.scatter(adv_principal_curvatures[1, ...].mean(axis=(0, 1)),
           s=0.005, alpha=1.0, c=plot_colors[1])

ax.format(
    title=f'Curvature profile, averaged across {num_images} images',
    xlim=(-5, num_dims+5),
    ylabel='Curvature',
    xlabel='Principal curvature direction',
    grid=False
)
for ax_loc in ['top', 'right']:
    ax.spines[ax_loc].set_color('none')
pplt.show()

#fig.savefig('../data/curvature_profile.png', transparent=True, bbox_inches='tight', pad_inches=0.01, dpi=dpi)

In [ ]:
# TODO:
# y & x axis should be the same scale
# no need to have a dot on the y axis, should use an arrow instead
num_plot_images = 3
offset = 0

for model_idx, (model_, data_) in enumerate(zip([model_natural, ], [data_natural, ])):
    num_models, num_images, num_advs, num_directions, num_pixels = adv_principal_directions.shape
    
    fig, axs = pplt.subplots(nrows=3, ncols=num_plot_images, figwidth=figwidth, dpi=dpi, hspace=2, wspace=2)
    axs.format(
        ylabel = 'Principal curvature direction'
    )
    for image_idx in range(offset, offset+num_plot_images):
        adv_idx = np.random.randint(low=0, high=num_advs)
        most_flat_index = np.argmin(np.abs(adv_principal_curvatures[model_idx, image_idx, adv_idx, :]))
        curvature_indices = [0, most_flat_index, -1]
        dataset_image_idx = adv_origin_indices[model_idx, image_idx]
        origin = data_['images'][dataset_image_idx, ...]

        boundary_image = get_paired_boundary_image(
            model=model_,
            origin=origin,
            alt_image=data_['advs'][dataset_image_idx, adv_idx, ...],
            num_steps_per_iter=num_steps_per_iter,
            num_iters=num_iters
        )[0]
        boundary_dist = np.linalg.norm(boundary_image.reshape(-1) - origin.reshape(-1))

        for ax_idx, curvature_idx in enumerate(curvature_indices):
            ax = axs[ax_idx, image_idx-offset]
            adv1 = boundary_image.reshape(-1)
            principal_direction = adv_principal_directions[model_idx, image_idx, adv_idx, :, curvature_idx]
            # Source of error?
            adv2 = origin.reshape(-1) + boundary_dist * principal_direction
            dec_advs, labels = pl.plot_dec_space(origin[None, ...], adv1, adv2, model_, offset=1.0,
                              n_grid=100, len_grid_scale=1.8, show_legend=False, show_advs=True,
                              overlay_inbounds=True, ax=ax)
            ax.legend(handles=labels, loc='upper left', ncols=1, title='predicted\nclass')
            ax.format(
                title=f'Curvature = {adv_principal_curvatures[model_idx, image_idx, adv_idx, curvature_idx]:.5f}',
                xlabel=f'Adversarial direction number {adv_idx}',
            )
    plt.show()
    #fig.savefig(f'../data/curvature_visualizations_1.pdf', transparent=True, bbox_inches='tight', pad_inches=0.01, dpi=dpi)

In [ ]:
fig, axs = pplt.subplots(nrows=1, ncols=2, figwidth=figwidth_inch, dpi=dpi)

percentiles = np.percentile(rand_subspace_pcs[0, ...], (5, 95), axis=0)
std = np.std(rand_subspace_pcs[0, ...], axis=0)

axs[0].scatter(rand_subspace_pcs[0, ...].mean(axis=0), s=2.0, c=plot_colors[0],
               bardata=np.std(rand_subspace_pcs[0, ...], axis=0), barc=plot_colors[0], barlw=0.5, capsize=0.0,)
axs[0].scatter(rand_subspace_pcs[1, ...].mean(axis=0), s=2.0, c=plot_colors[1],
               bardata=np.std(rand_subspace_pcs[1, ...], axis=0), barc=plot_colors[1], barlw=0.5, capsize=0.0,)
axs[0].axhline(0.0, color='black', linestyle='dashed', linewidth=0.5)
axs[0].format(
    title=f'Random subspaces'
)
for ax_loc in ['top', 'right']:
    axs[0].spines[ax_loc].set_color('none')

axs[1].scatter(adv_subspace_pcs[0, ...].mean(axis=0), s=2.0, c=plot_colors[0],
               bardata=np.std(adv_subspace_pcs[0, ...], axis=0), barc=plot_colors[0], barlw=0.5, capsize=0.0,)
axs[1].scatter(adv_subspace_pcs[1, ...].mean(axis=0), s=2.0, c=plot_colors[1],
               bardata=np.std(adv_subspace_pcs[1, ...], axis=0), barc=plot_colors[1], barlw=0.5, capsize=0.0,)
axs[1].axhline(0.0, color='black', linestyle='dashed', linewidth=0.5)
axs[1].format(
    title=f'Adversarial subspaces'
)
for ax_loc in ['top', 'right']:
    axs[1].spines[ax_loc].set_color('none')

axs.format(
    ylabel='Curvature',
    xlabel='Principal curvature directions',
    grid=False
)

legend_handles = [mpatches.Patch(color=plot_colors[0], label='Natural'),
                  mpatches.Patch(color=plot_colors[1], label='Adversarial')]
axs[0].legend(handles=legend_handles, loc='upper right', ncols=1, frame=False)

pplt.show()

fig.savefig(f'../data/subspace_curvatures.png', transparent=True, bbox_inches='tight', pad_inches=0.01, dpi=dpi)

In [ ]:
import svgutils.compose as sc
from IPython.display import SVG, Image 

In [ ]:
sc.Figure(figwidth, figwidth, 
    sc.Panel(sc.SVG('../data/mean_curvature_boxplots.svg')),
    sc.Panel(sc.SVG('../data/subspace_curvatures.svg')),
    sc.Panel(sc.SVG('../data/curvature_profile.svg'))
    ).save('compose.svg')
SVG('compose.svg')

In [ ]:
import xml.etree.ElementTree as etree
import re
from six import StringIO
import requests

import svgpathtools as svgpt
from svgpath2mpl import parse_path

In [ ]:
diagram = '../data/curvature_diagram.svg'
imported_diagram = SVG(diagram)

In [ ]:
imported_diagram

In [ ]:
curve_paths, curve_attributes, svg_attributes = svgpt.svg2paths2(diagram)

In [ ]:
svg_attributes

In [ ]:
curve_attributes[0]

In [ ]:
curve_attributes[3]

In [ ]:
def normalize_hex(c):
    if c.startswith('#') and len(c) == 4:
        return '#{0}{0}{1}{1}{2}{2}'.format(c[1], c[2], c[3])
    return c

paths = [parse_path(attrib['d']) for attrib in curve_attributes]
facecolors = [normalize_hex(attrib.get('fill', 'none')) for attrib in curve_attributes]
edgecolors = [normalize_hex(attrib.get('stroke', 'none')) for attrib in curve_attributes]
linewidths = [attrib.get('stroke_width', 1) for attrib in curve_attributes]
collection = mpl.collections.PathCollection(paths, 
                                      edgecolors=edgecolors, 
                                      linewidths=linewidths,
                                      facecolors=facecolors)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
collection.set_transform(ax.transData)
ax.add_artist(collection)
#ax.set_xlim([0, 960])
#ax.set_ylim([540, 0])

In [ ]:
paths = [parse_path(attrib['d']) for attrib in curve_attributes]
facecolors = [attrib.get('fill', 'none') for attrib in curve_attributes]
edgecolors = [attrib.get('stroke', 'none') for attrib in curve_attributes]
linewidths = [attrib.get('stroke_width', 1) for attrib in curve_attributes]
collection = mpl.collections.PathCollection(paths, 
                                      edgecolors=edgecolors, 
                                      linewidths=linewidths,
                                      facecolors=facecolors)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
collection.set_transform(ax.transData)
ax.add_artist(collection)
ax.set_xlim([0, 960])
ax.set_ylim([540, 0])

In [ ]:
r = requests.get('http://thenewcode.com/assets/images/thumbnails/homer-simpson.svg')
tree = etree.parse(StringIO(r.text))
root = tree.getroot()
width = int(re.match(r'\d+', root.attrib['width']).group())
height = int(re.match(r'\d+', root.attrib['height']).group())
path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')
paths = [parse_path(elem.attrib['d']) for elem in path_elems]
facecolors = [elem.attrib.get('fill', 'none') for elem in path_elems]
edgecolors = [elem.attrib.get('stroke', 'none') for elem in path_elems]
linewidths = [elem.attrib.get('stroke_width', 1) for elem in path_elems]
collection = mpl.collections.PathCollection(paths, 
                                      edgecolors=edgecolors, 
                                      linewidths=linewidths,
                                      facecolors=facecolors)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
collection.set_transform(ax.transData)
ax.add_artist(collection)
ax.set_xlim([0, width])
ax.set_ylim([height, 0])